In [ ]:
from fastai.text import *
import numpy as np
import json

%matplotlib inline

## Let's Set All The Paths? 
Let's set all the path which is required to work on our data.

* `path` is the root input directory.
* `outputPath` is where we would be saving our csv , models etc.
* `datasetPath` is the directory to the dataset.

In [ ]:
inputPath = Path('/kaggle/input')
outputPath = Path('/kaggle/working')
#datasetPath = path/'covid19-tweets/'

In [ ]:
inputPath.ls()

In [ ]:
!pip install twarc

In [ ]:
consumer_key= "3AO7qGIh76LUgZgcXs0VNWtjC"
consumer_secret= "DJFneehCiWKbLbqqFfBeYZuu1WbRoFqd2LEL9VTeMHGwRCkc5U"
access_token= "804532808190291969-ON0c25lsbJg26regleTnEb1SypMhLk7"
access_token_secret= "yN6R535MAD3kf4tlhU2s6DGP2CCVosbJZfm7mxgCSjjtY"

In [ ]:
import gzip
import json

from tqdm import tqdm
from twarc import Twarc
from pathlib import Path

twarc = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)
#data_dirs = ['2020-01', '2020-02','2020-03', '2020-04']
data_dirs = ['2020-01']


def main():
    for data_dir in data_dirs:
        for path in Path(inputPath/data_dir).iterdir():
            if path.name.endswith('.txt'):
                hydrate(path, data_dir)


def _reader_generator(reader):
    b = reader(1024 * 1024)
    while b:
        yield b
        b = reader(1024 * 1024)


def raw_newline_count(fname):
    """
    Counts number of lines in file
    """
    f = open(fname, 'rb')
    f_gen = _reader_generator(f.raw.read)
    return sum(buf.count(b'\n') for buf in f_gen)


def hydrate(id_file, data_dir):
    print('hydrating {}'.format(id_file))

    #gzip_path = id_file.with_suffix('.jsonl.gz')
    gzip_path = (outputPath/id_file.name).with_suffix('.jsonl.gz')
    if gzip_path.is_file():
        print('skipping json file already exists: {}'.format(gzip_path))
        return

    num_ids = raw_newline_count(id_file)

    with gzip.open(gzip_path, 'w') as output:
        with tqdm(total=num_ids) as pbar:
            for tweet in twarc.hydrate(id_file.open()):
                output.write(json.dumps(tweet).encode('utf8') + b"\n")
                pbar.update(1)


if __name__ == "__main__":
    main()

some of the code ceels are converted to markdown to prevent htme from getting executed in the committed kernel. These cells should be converted back to code when this kernel would be forked for further developement.

# load_files(fileNames: tuple):
# fileList = []

# for file in fileNames:
#     fileList.append(file)
#     
# return fileList

# uncomment this line if you want both folders.
#files = load_files((pmcCustomLicense,biorxivMedrxiv))
#files = load_files((biorxivMedrxiv.iterdir()))

type(files)

To check the sample of the contents of the processed list run the following code.

files[:3]

The file paths processed with the `load_files()` contains path objects and may not be suitable for certain operations. For example path objects can't be iterated over. 

We will convert those path objects into their string "paths" with the `filePath()` function. This functions does the following -->
* Takes in the list havin gthe path objects.
* Iterates through the list and converts each list item i.e. the path objects into string format.
* Then it puts those string paths into the `filePath` list.

def filePath(files):
    filePath = []
    
    for file in files:
        filePath.append(str(file))
        
    return filePath

filePaths=filePath(files)

The `getRawFiles()` functions does the following -->
* Takes in a list of file paths
* Goes through each file path from the list `files` and uses the `json.load()` method to read the contents.
* finally it appends these json object into the `rawFiles` list.

def getRawFiles(files: list):
    rawFiles = []
        
    for fileName in files:
            rawFile = json.load(open(fileName, 'rb'))
            rawFiles.append(rawFile)
            
    return rawFiles

In [ ]:
rawFiles = getRawFiles(filePaths)


`format_name()`-->
* Takes in the json object with author as the key.
* Joins the names in the following sequence <first name> <middle name> <last name.
* Function also takes care o fthe fact that if there no middle name then just do <first name> <last name

`format_affiliation` -->
* Takes in the json object with key as affiliation.
* If location details are there in json then put it into a list and return it.
* If institution is there then join location and instituion details together and put it into a list.

`format_authors()` -->
* Takes in the json object with key as author.
* Joins the author's name with the affiliations if affiliations are available.

`format_body()` -->
* Takes in the json object with key as body_text.
* Extracts the text and then appends it into a list.

`format_bib()` -->
* Takes in the json object with key as bib.
* Joins the 'title', 'authors', 'venue', 'year' together to form a string.

def format_name(author):
    middle_name = " ".join(author['middle'])
    
    if author['middle']:
        return " ".join([author['first'], middle_name, author['last']])
    else:
        return " ".join([author['first'], author['last']])


def format_affiliation(affiliation):
    text = []
    location = affiliation.get('location')
    if location:
        text.extend(list(affiliation['location'].values()))
    
    institution = affiliation.get('institution')
    if institution:
        text = [institution] + text
    return ", ".join(text)

def format_authors(authors, with_affiliation=False):
    name_ls = []
    
    for author in authors:
        name = format_name(author)
        if with_affiliation:
            affiliation = format_affiliation(author['affiliation'])
            if affiliation:
                name_ls.append(f"{name} ({affiliation})")
            else:
                name_ls.append(name)
        else:
            name_ls.append(name)
    
    return ", ".join(name_ls)

def format_body(body_text):
    texts = [(di['section'], di['text']) for di in body_text]
    texts_di = {di['section']: "" for di in body_text}
    
    for section, text in texts:
        texts_di[section] += text

    body = ""

    for section, text in texts_di.items():
        body += section
        body += "\n\n"
        body += text
        body += "\n\n"
    
    return body

def format_bib(bibs):
    if type(bibs) == dict:
        bibs = list(bibs.values())
    bibs = deepcopy(bibs)
    formatted = []
    
    for bib in bibs:
        bib['authors'] = format_authors(
            bib['authors'], 
            with_affiliation=False
        )
        formatted_ls = [str(bib[k]) for k in ['title', 'authors', 'venue', 'year']]
        formatted.append(", ".join(formatted_ls))

    return "; ".join(formatted)

`generate_clean_df()` -->
* Uses the above helper functions to create a pandas dataframe.
* The resulting dataframe would have the following columns ->
  *'paper_id'
  * 'title'
  * 'authors'
  * 'affiliations'
  * 'abstract'
  * 'text'
  * 'bibliography'
  * 'raw_authors'
  * 'raw_bibliography'

def generate_clean_df(all_files):
    cleaned_files = []
    
    for file in all_files:
        features = [
            file['paper_id'],
            file['metadata']['title'],
            format_authors(file['metadata']['authors']),
            format_authors(file['metadata']['authors'], 
                           with_affiliation=True),
            format_body(file['abstract']),
            format_body(file['body_text']),
            format_bib(file['bib_entries']),
            file['metadata']['authors'],
            file['bib_entries']
        ]

        cleaned_files.append(features)

    col_names = ['paper_id', 'title', 'authors',
                 'affiliations', 'abstract', 'text', 
                 'bibliography','raw_authors','raw_bibliography']

    clean_df = pd.DataFrame(cleaned_files, columns=col_names)
    clean_df.head()
    
    return clean_df

pd.set_option('display.max_columns', None) 
cleanDf = generate_clean_df(rawFiles)
cleanDf.head()

Finally save the dataframe to a csv file. Since the entire previous process takes a bit of time, so saving the file to csv saves time later.

cleanDf.to_csv(outputPath/'cleandf.csv')

## The Data 
We then create a databunch from the csv that we created from the json files.

The `createDataBunchForLanguageModel()` is a helper function. This does the following -->
* Creates a databunch from the csv file.
* The data in the databunch is fetched from the 'text' column of the csv file.
* 10% of the data is reserved as the validation data.
* Since the language model that we will create down the line will be a self-supervised model. It takes in the label from teh data itself.
* Thus the `label_for_lm()` helps us with that.
* After the databunch is created, it is saved to a pickle file. Next time when this notebook is run then we don't need to go through the process of creation of the databunch once again. We can just use the pickle file.

def createDataBunchForLanguageModel(outputPath: Path,
                                    csvName: str,
                                    textCol: str, 
                                    pickelFileName: str,
                                    splitBy: float, 
                                   batchSize: int):
    data_lm = TextList.from_csv(outputPath,
                                f'{csvName}.csv',
                                cols=textCol)\
                  .split_by_rand_pct(splitBy)\
                  .label_for_lm()\
                  .databunch(bs=batchSize)
    
    data_lm.save(f'{pickelFileName}.pkl')

createDataBunchForLanguageModel(outputPath,
                                    'cleandf',
                                    'text', 
                                    'cleanDf',
                                    0.1,
                                    48)

The `loadData()` is a helper function to load the databunch file. It takes in the required batch size which we want to load from the databunch.

def loadData(outputPath: Path,
             databunchFileName: str,
             batchSize: int,
             showBatch: bool= False):
    
    data_lm = load_data(outputPath,
                       f'{databunchFileName}.pkl',
                       bs=batchSize)
    
    if showBatch:
        data_lm.show_batch()
        
    return data_lm

## Building The Learner 
We use ULMFIT to create a language model on the research data corpus and then fine tune this language model.

We create the language model learner. The `language_model_learner()` is a fastai method which helps us to build a learner object with the data created in the previous sections. 

Here we use a batch size of 48. This learner is created with a pretrained language model architecure `AWD_LSTM`. This is the self supervised part. This is a model which was trained on a english language corpus to predict the next sequence of sentences and thus understands the structure of the language.

We just need to fine tune this modle on our data corpus which then can be used to build a classifier.

I am not buidling any classifier as yet because I don't have any idea as to what we need to classify. However I believe that this fine tuned language modle can be customized to find hidden information in the large corpus of research papers which would otherwise be hidden/missed by the human readers.

learner = language_model_learner(loadData(outputPath,
             'cleanDf',
             48,
             showBatch= False),
             AWD_LSTM,
             drop_mult=0.3)

We plot the learning rate for our language model. From this plot we will try to find the best learnign rate suitable for our model. 
The `plotLearningRate()` is a helper function which plots the learnign rate for us.

def plotLearningRate(learner, skip_end):
    learner.lr_find()
    learner.recorder.plot(skip_end=skip_end)

plotLearningRate(learner, 15)

We then take in that learning rate as our start from where the plot diverges. The model is then trained with the "fit one cycle" with this strating learning rate.

This is where we train the head only.

learner.fit_one_cycle(1, 1e-02, moms=(0.8,0.7))

Since we are creating a language model, we are not overly concerned about getting the best possible accuracy here. So, we unfreeze the network and train some more.

learner.unfreeze()
learner.fit_one_cycle(10,1e-3, moms=(0.8,0.7))

Finally we save the fine tuned language model for use in testing/prediction.

learner.save('fineTuned')

## Testing 
Let's see if the model can connect the information/knowledge from the corpus. We first load the saved model and then try to find prediction for a search term.

In [ ]:
learner = learner.load('fineTuned')

In [ ]:
TEXT = "Range of incubation periods for the disease in humans"
N_WORDS = 40
N_SENTENCES = 2

In [ ]:
print("\n".join(learner.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

We have a pretty good language model available now. Now, that we have our language modle fine tuned, we can save the modle which predicts the next sequence of words and the encoder which is responsible for creating and updating the hidden states.

We will use `.save()` to save the `.pth` file.

def save(learner,
         saveEncoder: bool = True):
    
    if saveEncoder:
        learner.save_encoder('fine_tuned_encoder')
        
    learner.save('fine_tuned_model')

save(learner)

## End Notes 
I may not have completed any dataset tasks with this kernel but I hope that this kernel and the language model built in this kernel would be helpful for someone else who might be working on the dataset tasks. This might ultimately help us to gain some insights into the research and this could ultimately help us to fight this dreadful disease.

I am in no way an expert in NLP so the network that I have developed here is based on the code from the lesson3 of the course - ["Practicel Deep Learning For coders" by fastai](https://www.fast.ai/2019/01/24/course-v3/). 

I am hoping that someone who is more seasoned in NLP and deep learning would improve on this model and bring out something useful which could then contibute towards fighting COVID-19.